# Error fields
This notebook illustrates the error field computation.

In [ ]:
using DIVAnd
#push!(LOAD_PATH,"c:/users/jmbeckers/Documents/Github/DIVAnd.jl/src","."); using DIVAnd
using PyPlot
using Dates
using Statistics
using LinearAlgebra

## Data reading
Download the file (it not already done) and read it.

In [ ]:
varname = "Salinity"
filename = "../data/WOD-Salinity-Provencal.nc"

if !isfile(filename)    
    download("https://dox.ulg.ac.be/index.php/s/PztJfSEnc8Cr3XN/download",filename)
else 
    @info "File already downloaded"
end

In [ ]:
obsval,obslon,obslat,obsdepth,obstime,obsid = loadobs(Float64,filename,"Salinity");

## Topography and grid definition
See the notebook on [bathymetry](../2-Preprocessing/06-topography.ipynb) for more explanations.

Define domain and resolution, create the domain.

In [ ]:
dx = dy = 0.125/2.
lonr = 2.5:dx:12.
latr = 42.3:dy:44.6

mask,(pm,pn),(xi,yi) = DIVAnd_rectdom(lonr,latr);

Download the bathymetry file and load it.

In [ ]:
bathname = "../data/gebco_30sec_4.nc"
if !isfile(bathname)
    download("https://dox.ulg.ac.be/index.php/s/RSwm4HPHImdZoQP/download",bathname)
else
    @info("Bathymetry file already downloaded")
end

In [ ]:
bx,by,b = load_bath(bathname,true,lonr,latr);

Create a land-sea mask based on the bathymetry.

In [ ]:
mask = falses(size(b,1),size(b,2))

for j = 1:size(b,2)
    for i = 1:size(b,1)
        mask[i,j] = b[i,j] >=1.0
    end
end

## Data selection for example

Cross validation, error calculations etc. assume independant data. Hence do not take high-resolution vertical profiles with all data but restrict yourself to specific small depth range. Here we limit outselves to August data at surface:

In [ ]:
sel = (obsdepth .< 1) .& (Dates.month.(obstime) .== 8)

obsval = obsval[sel]
obslon = obslon[sel]
obslat = obslat[sel]
obsdepth = obsdepth[sel]
obstime = obstime[sel]
obsid = obsid[sel];
@show (size(obsval))
checkobs((obslon,obslat,obsdepth,obstime),obsval,obsid)

### Analysis
Analysis parameters have been calibrated in the other notebook [13-processing-parameter-optimization example.ipynb](13-processing-parameter-optimization). 

⚠ if the statistical parameters are incorrectly estimated, the error fields are meaningless and only provide an idea of data coverage.

The analysis parameters are:

In [ ]:
len = 0.3
epsilon2 = 1.0;

Analysis `fi` using mean data as background.    
Structure `s` is stored for later use in error calculation.

In [ ]:
fi, s = DIVAndrun(mask,(pm,pn),(xi,yi),(obslon,obslat),obsval.-mean(obsval),len,epsilon2);

Create a simple plot of the analysis

In [ ]:
pcolor(xi,yi,fi.+mean(obsval),vmin=37,vmax=38.5);
colorbar(orientation="horizontal")
contourf(bx,by,copy(b'), levels = [-1e5,0],colors = [[.5,.5,.5]])
aspectratio = 1/cos(mean([ylim()...]) * pi/180)
gca().set_aspect(aspectratio)

## Exact error and approximations

Details can be found in the publication:

Approximate and Efficient Methods to Assess Error Fields in Spatial Gridding with Data Interpolating Variational Analysis (DIVA) Beckers, Jean-Marie; Barth, Alexander;  Troupin, Charles, Alvera-Azcarate, A.  *Journal of Atmospheric & Oceanic Technology* (2014), **31(2)**, 515-530     
https://orbi.uliege.be/handle/2268/161069      
https://journals.ametsoc.org/doi/abs/10.1175/JTECH-D-13-00130.1

In the 2D case you can try to calculate the exact error expression. This demands the computationally expensive evaluation of `diag(s.P)` accessible via the analysis returned structure `s`. This is only available with `DIVAndrun`.

In [ ]:
# plots the error field `exerr`
function ploterr(exerr; vmin=0, vmax=1.5, cmap="hot_r")
    pcolor(xi,yi,exerr,vmin=vmin, vmax=vmax, cmap=cmap);
    colorbar(orientation="horizontal")
    contourf(bx,by,copy(b'), levels = [-1e5,0],colors = [[.5,.5,.5]])
    plot(obslon, obslat, "k.", markersize=.5)
    ylim(extrema(yi))
    gca().set_aspect(1/cos(mean([ylim()...]) * pi/180))
end

In [ ]:
exerr, = statevector_unpack(s.sv,diag(s.P),NaN)
ploterr(exerr)
title("Error using P, scaled by global background variance");

In [ ]:
aerrora,method = DIVAnd_errormap(mask,(pm,pn),(xi,yi),(obslon,obslat),obsval.-mean(obsval),len,epsilon2,
    s;
    method = "exact",
    Bscale = false)

In [ ]:
aerrora[1]

In [ ]:
ploterr(aerrora[1])
title("Error using automatic version $method");

In [ ]:
aerror,method = DIVAnd_errormap(mask,(pm,pn),(xi,yi),(obslon,obslat),obsval.-mean(obsval),len,epsilon2,
    s;
    method = "auto",
    Bscale = false)
ploterr(aerror)
title("Error using automatic version $method");

Do you see any difference between the exact and clever poor man's error ? 
## Difference between error fields
We also overlay the data positions.

In [ ]:
ploterr(aerror-exerr,vmin=-0.2, vmax=0.2, cmap="RdBu_r")
title("Error on error");

## How to plot standart deviations ?

Error fields shown above are error variance divided by a global background error variance $\sigma^2$. The latter is difficult to assess but a simple practical way is to use data variance and split it into background error variance and observational error variance $\epsilon^2$, assuming the $epsilon2$ value is correct:

$epsilon2$ = ${ \epsilon^2 \over \sigma^2 } $

$ \epsilon^2 + \sigma^2 $ = var(obsval)

Provides

$\sigma^2$= $ {1 \over 1 + epsilon2 } $ var(obsval)


If you are not sure about the value of $epsilon2$ you might consider using DIVAnd_adaptedeps2(s, fi) 

In [ ]:
epsilon2=DIVAnd_adaptedeps2(s, fi)

In [ ]:
myabserror=sqrt.(1.0/(1.0+epsilon2)*var(obsval).*exerr)
ploterr(myabserror)
title("Standard deviation");

## How to calculate error on average fields

In [ ]:
# uniform grid is assumed
gridsurf=ones(Float64,size(s.P)[1],1)

# "volumes" for integration
function volint(mask, pmn)
 NDIM = ndims(mask)
    dim = size(mask)
    # Utility array holding volume based on metrics
    volume = zeros(Float64, dim)
    volume[mask] .= 1.0
    for i = 1:NDIM
        volume .= volume ./ pmn[i]
    end
    return volume
end
gridsurf=statevector_pack(s.sv,(volint(mask,(pm,pn)),))
gridsurf=reshape(gridsurf, length(gridsurf), 1)

erronmean=diagMtCM(s.P,gridsurf)[1]/((sum(gridsurf))^2)
# That is the error variance of the mean, still scaled by the background error variance (no units)






In [ ]:
gridsurf

In [ ]:
s.P

In [ ]:

# now scale by estimate of $\sigma^2$ and take square root

myabserror=sqrt(1.0/(1.0+epsilon2)*var(obsval)*erronmean)


In [ ]:
mean(fi[.!isnan.(fi)])

In [ ]:
mean(obsval)